Initial imports

In [1]:
from qiskit_research.protein_folding.interactions.random_interaction import (
    RandomInteraction,
)
from qiskit_research.protein_folding.interactions.miyazawa_jernigan_interaction import (
    MiyazawaJerniganInteraction,
)
from qiskit_research.protein_folding.peptide.peptide import Peptide
from qiskit_research.protein_folding.protein_folding_problem import (
    ProteinFoldingProblem,
)

from qiskit_research.protein_folding.penalty_parameters import PenaltyParameters

from qiskit.utils import algorithm_globals, QuantumInstance

algorithm_globals.random_seed = 23

Main chain declaration

In [2]:
possible_chains = ["APRLRFY","RPKPQQF","RPKPQQFFGLM","YFLRIPKWDNQ"]
main_chain = possible_chains[0] 
side_chain = side_chains = [""] * len(main_chain)



Interaction models

In [3]:
random_interaction = RandomInteraction()
mj_interaction = MiyazawaJerniganInteraction()

Physical constraints

In [4]:
penalty_back = 10
penalty_chiral = 10
penalty_1 = 10

penalty_terms = PenaltyParameters(penalty_chiral, penalty_back, penalty_1)

Peptide

In [5]:
peptide = Peptide(main_chain, side_chains)

Definition of problem

In [6]:
n_probs = 100
qubit_ops = []
#I set "random interactions" because my goal is to see how "close" we get to the classic value, so I need different inputs
for i in range(n_probs):
    protein_folding_problem = ProteinFoldingProblem(peptide, random_interaction, penalty_terms)
    qubit_op = protein_folding_problem.qubit_op()
    qubit_ops.append(qubit_op)


VQE

In [0]:
from qiskit.circuit.library import RealAmplitudes, ExcitationPreserving, EfficientSU2, PauliTwoDesign 
from qiskit.algorithms.optimizers import COBYLA, GradientDescent, SPSA, SLSQP
from qiskit.algorithms import NumPyMinimumEigensolver
from qiskit.algorithms.minimum_eigensolvers import SamplingVQE
from qiskit import execute, Aer
from qiskit.primitives import Sampler
import numpy as np
from sklearn.metrics import mean_squared_error

import warnings

n_iter = 100

# set classical optimizer
optimizers = [COBYLA(maxiter=n_iter), GradientDescent(maxiter=n_iter), SPSA(maxiter=n_iter), SLSQP(maxiter=n_iter)]

# set variational ansatz
ansaetze = [RealAmplitudes(reps=1), ExcitationPreserving(reps=1), EfficientSU2(reps=1), PauliTwoDesign]

counts = []
values = []


def store_intermediate_result(eval_count, parameters, mean, std):
    counts.append(eval_count)
    values.append(mean)

#changing warning management
np.seterr(all='warn')
warnings.filterwarnings('error')

#calculate classic eigenvalues
classic_eigs = []
for q in qubit_ops:
    matrix = np.asarray(q.to_matrix())
    eig = np.min(np.real(np.linalg.eigvals(matrix)))
    #print(eig)
    classic_eigs.append(eig)


for i in range(len(ansaetze)):
    ansatz = ansaetze[i]
    for j in range(len(optimizers)):
        optimizer = optimizers[j]
        print("ANSATZ: " + str(ansatz.name) + " OPTIMIZER: " + str(optimizer.__class__.__name__))
# initialize VQE using CVaR with alpha = 0.1
        vqe = SamplingVQE(
            Sampler(),
            ansatz=ansatz,
            optimizer=optimizer,
            aggregation=0.1,
            callback=store_intermediate_result,
            )
        try:
            current_results = []
            for q in qubit_ops:
                raw_result = vqe.compute_minimum_eigenvalue(q)
                current_results.append(raw_result.eigenvalue)
            #print(current_results)
            print("MSE: " + str(mean_squared_error(classic_eigs, current_results)))
        except:
            print('NAN detected!')
            

ANSATZ: RealAmplitudes OPTIMIZER: COBYLA
MSE: 0.004343551673878294
ANSATZ: RealAmplitudes OPTIMIZER: GradientDescent
MSE: 0.7732675721858836
ANSATZ: RealAmplitudes OPTIMIZER: SPSA
MSE: 28.90107650377034
ANSATZ: RealAmplitudes OPTIMIZER: SLSQP
NAN detected!
ANSATZ: ExcitationPreserving OPTIMIZER: COBYLA
MSE: 452.2934439041089
ANSATZ: ExcitationPreserving OPTIMIZER: GradientDescent
MSE: 452.2934439041089
ANSATZ: ExcitationPreserving OPTIMIZER: SPSA
NAN detected!
ANSATZ: ExcitationPreserving OPTIMIZER: SLSQP
MSE: 452.2934439041089
ANSATZ: EfficientSU2 OPTIMIZER: COBYLA
MSE: 0.004184682815476655
ANSATZ: EfficientSU2 OPTIMIZER: GradientDescent
MSE: 0.9931693843220469
ANSATZ: EfficientSU2 OPTIMIZER: SPSA
MSE: 417.64352923862776
ANSATZ: EfficientSU2 OPTIMIZER: SLSQP
NAN detected!


AttributeError: type object 'PauliTwoDesign' has no attribute 'name'

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=ddc54cc5-de90-4346-ada1-4ff24feba045' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>